# Project Zeus
## Created by Michael Wentz

### Import Modules

In [1]:
import numpy as np
import pandas as pd
import math

from bokeh.io import show, output_notebook, push_notebook
from bokeh.layouts import widgetbox
from bokeh.models import LinearAxis, Range1d, DataRange1d, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.models.widgets import Slider, RadioButtonGroup
from ipywidgets import interactive

### Load production data from excel file

In [2]:
file = 'https://raw.githubusercontent.com/mwentzWW/petrolpy/master/Zeus/Sample_Production/Sample_Prod_Data.xlsx'
well_data = pd.read_excel(file)
well_data.columns = [c.lower() for c in well_data.columns]
well_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 5 columns):
api number         704 non-null object
production date    704 non-null datetime64[ns]
oil_bbl            694 non-null float64
gas_mcf            694 non-null float64
water_bbl          693 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 24.8+ KB


### Check Data Layout

In [3]:
well_data.head()

,api number,production date,oil_bbl,gas_mcf,water_bbl
0,40 063 05037,1959-05-31,3533.0,0.0,0.0
1,40 063 05037,1959-06-30,4763.0,0.0,0.0
2,40 063 05037,1959-07-31,4475.0,1570.0,0.0
3,40 063 05037,1959-08-31,2620.0,917.0,0.0
4,40 063 05037,1959-09-30,3531.0,1236.0,0.0


### Delete API because there is only one well, and we don't need water

In [4]:
well_data.drop(columns=['api number', 'water_bbl'], inplace=True)
well_data.head()

,production date,oil_bbl,gas_mcf
0,1959-05-31,3533.0,0.0
1,1959-06-30,4763.0,0.0
2,1959-07-31,4475.0,1570.0
3,1959-08-31,2620.0,917.0
4,1959-09-30,3531.0,1236.0


### Replace NaN with zeros for computations

In [5]:
well_data.fillna(value=0, inplace=True)

### Normalize data by time

In [6]:
num_months = len(well_data['oil_bbl'])
num_months

704

In [7]:
months = []
for x in range(0, num_months):
    months.append(x)
months[0:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [8]:
well_data['time'] = months
print(well_data.head())

  production date  oil_bbl  gas_mcf  time
0      1959-05-31   3533.0      0.0     0
1      1959-06-30   4763.0      0.0     1
2      1959-07-31   4475.0   1570.0     2
3      1959-08-31   2620.0    917.0     3
4      1959-09-30   3531.0   1236.0     4


### Calculate the type curve for oil and gas

In [9]:
print(well_data.head())

  production date  oil_bbl  gas_mcf  time
0      1959-05-31   3533.0      0.0     0
1      1959-06-30   4763.0      0.0     1
2      1959-07-31   4475.0   1570.0     2
3      1959-08-31   2620.0    917.0     3
4      1959-09-30   3531.0   1236.0     4


In [10]:
print(well_data.loc[[3], ('gas_mcf')])

3    917.0
Name: gas_mcf, dtype: float64


In [11]:
def plot_oil(qi_oil= well_data.oil_bbl.max(), b_value=0.8, initial_decline_rate=15, start_of_exp=well_data.time.max(), exp_yes_no=False, exp_decline_rate=10):
    # Decline rates are input as annual whole numbers, then converted to decimal monthly numbers
    initial_decline_rate = (initial_decline_rate/100)/12
    exp_decline_rate = (exp_decline_rate/100)/12
    
    well_data['type_curve_oil'] = qi_oil*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    if exp_yes_no == True:
        start_of_exp = int(start_of_exp)
        well_data['type_curve_expon'] = well_data.loc[start_of_exp, ('type_curve_oil')]*2.71828**(-exp_decline_rate*(well_data.time-start_of_exp))
        well_data.loc[start_of_exp:, ('type_curve_oil')] = 0
        well_data.loc[0:start_of_exp-1, ('type_curve_expon')] = 0
        connector = well_data.loc[start_of_exp, ('type_curve_expon')]
        well_data.type_curve_oil = well_data.type_curve_oil + well_data.type_curve_expon
        well_data.type_curve_expon.astype('float')
    well_data.type_curve_oil.astype('float')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_oil > qi_oil] = qi_oil
    
    p = figure(title= 'Oil Type Curve', y_axis_type='log')
    p.line(well_data.time, well_data.oil_bbl, line_color= 'green', line_width= 2, legend= 'Oil Production')
    p.line(well_data.time, well_data.type_curve_oil, line_color= 'black', line_width= 2, line_dash= 'dashed', legend= 'Curve Fit')
    
    
    
    p.yaxis.axis_label = 'BOPM'
    p.xaxis.axis_label = 'MONTH'
    p.y_range.start = 10
    p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p.title.align = 'center'
    
    
    show(p, notebook_handle=True)


def plot_gas(qi_gas= well_data.gas_mcf.max(), b_value=1.8, initial_decline_rate=15, start_of_exp=well_data.time.max(), exp_yes_no=False, exp_decline_rate=10):
    # Decline rates are input as annual whole numbers, then converted to decimal monthly numbers
    initial_decline_rate = (initial_decline_rate/100)/12
    exp_decline_rate = (exp_decline_rate/100)/12
    
    well_data['type_curve_gas'] = qi_gas*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    if exp_yes_no == True:
        start_of_exp = int(start_of_exp)
        well_data['type_curve_expon'] = well_data.loc[start_of_exp, ('type_curve_oil')]*2.71828**(-exp_decline_rate*(well_data.time-start_of_exp))
        well_data.loc[start_of_exp:, ('type_curve_gas')] = 0
        well_data.loc[0:start_of_exp-1, ('type_curve_expon')] = 0
        connector = well_data.loc[start_of_exp, ('type_curve_expon')]
        well_data.type_curve_gas = well_data.type_curve_gas + well_data.type_curve_expon
        well_data.type_curve_expon.astype('float')
    well_data.type_curve_oil.astype('float')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_gas > qi_gas] = qi_gas
    
    p = figure(title= 'Gas Type Curve', y_axis_type='log')
    p.line(well_data.time, well_data.gas_mcf, line_color= 'red', line_width= 2, legend= 'Gas Production')
    p.line(well_data.time, well_data.type_curve_gas, line_color= 'black', line_width= 2, line_dash= 'dashed', legend= 'Curve Fit')
    
    
    
    p.yaxis.axis_label = 'MCFPM'
    p.xaxis.axis_label = 'MONTH'
    p.y_range.start = 10
    p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p.title.align = 'center'
    
    
    show(p, notebook_handle=True)

### Plot production with type curves

In [12]:
output_notebook()

Loading BokehJS ...

In [13]:
def update_oil(b_value, initial_decline_rate, start_of_exp, exp_yes_no, exp_decline_rate):
    plot_oil(qi_oil= well_data.oil_bbl.max(), b_value=b_value, initial_decline_rate=initial_decline_rate, start_of_exp=start_of_exp, exp_yes_no=exp_yes_no, exp_decline_rate=exp_decline_rate)
    push_notebook()
    
def update_gas(b_value, initial_decline_rate, start_of_exp, exp_yes_no, exp_decline_rate):
    plot_gas(qi_gas= well_data.gas_mcf.max(), b_value=b_value, initial_decline_rate=initial_decline_rate, start_of_exp=start_of_exp, exp_yes_no=exp_yes_no, exp_decline_rate=exp_decline_rate)
    push_notebook()

In [14]:
interactive_plot_oil = interactive(update_oil, b_value= (0, 2, 0.01), initial_decline_rate= (0, 150), exp_yes_no = False, start_of_exp= (1, well_data.time.max(), 1), exp_decline_rate=(0, 20));
output = interactive_plot_oil.children[-1]
output.layout.height = '400px'
interactive_plot_oil

interactive(children=(FloatSlider(value=1.0, description='b_value', max=2.0, step=0.01), IntSlider(value=75, d…

In [15]:
interactive_plot_gas = interactive(update_gas, b_value= (0, 2, 0.01), initial_decline_rate= (0, 150), exp_yes_no = False, start_of_exp= (1, int(well_data.time.max()), 1), exp_decline_rate=(0, 20));
output = interactive_plot_gas.children[-1]
output.layout.height = '400px'
interactive_plot_gas

interactive(children=(FloatSlider(value=1.0, description='b_value', max=2.0, step=0.01), IntSlider(value=75, d…

In [16]:
print(well_data)

         production date  oil_bbl  gas_mcf   time  type_curve_oil  \
0    1959-05-31 00:00:00   3533.0      0.0    0.0     4763.000000   
1    1959-06-30 00:00:00   4763.0      0.0    1.0     4482.823529   
2    1959-07-31 00:00:00   4475.0   1570.0    2.0     4233.777778   
3    1959-08-31 00:00:00   2620.0    917.0    3.0     4010.947368   
4    1959-09-30 00:00:00   3531.0   1236.0    4.0     3810.400000   
..                   ...      ...      ...    ...             ...   
699  2017-09-30 00:00:00    682.0   7492.0  699.0      106.584615   
700  2017-10-31 00:00:00    618.0  13054.0  700.0      106.435754   
701  2017-11-30 00:00:00    720.0  13567.0  701.0      106.287308   
702  2017-12-31 00:00:00    608.0  13290.0  702.0      106.139276   
703  2018-01-31 00:00:00    501.0  11915.0  703.0      105.991655   

     type_curve_gas  
0      33356.000000  
1      31393.882353  
2      29649.777778  
3      28089.263158  
4      26684.800000  
..              ...  
699      746.4279